# rhino3dm exercise 02

## import and util funcs

In [1]:
import rhino3dm
from dataclasses import dataclass, field
import random
import string

print('rhino3dm ver: ', rhino3dm.__version__)

def generate_id() -> str:
    return "".join(random.choices(string.ascii_lowercase, k=12))

rhino3dm ver:  8.9.0

## read `.3dm` file

In [2]:
rhino_f_path = "./data/nongke.3dm"
file3dm_obj = rhino3dm.File3dm.Read(rhino_f_path)
print('rhino3dm.File3dm object: ', file3dm_obj)

rhino3dm.File3dm object:  <rhino3dm._rhino3dm.File3dm object at 0x1098086f0>

## read `notes`

In [3]:
file3dm_notes = rhino3dm.File3dm.ReadNotes(rhino_f_path).split(';')[:-1]

for idx, note in enumerate(file3dm_notes):
    print(
        note.strip().split(':')[0].strip(),
        ' : ',
        note.strip().split(':')[1].strip()
    )

name  :  nongke
lati  :  20
long  :  100

> **File3dm.ReadNotes()**
>
> [***method***
> `rhino3dm.File3dm.ReadNotes()`](https://mcneel.github.io/rhino3dm/python/api/File3dm.html#rhino3dm.File3dm.ReadNotes)
>
> <br />
>
> Reads only the notes from an existing 3dm file.
>
> <br />
>
> Parameters:
>
> -   path (str) – The file from which to read the notes.
>
> Returns:
>
> -   The 3dm file notes.
>
> Return type:
>
> -   str

## read `Layer`

In [4]:
layer_table = file3dm_obj.Layers
print('layer table object: ', layer_table)
print('num of layers: ', len(layer_table))
print()

for idx, layer in enumerate(iter(layer_table)):
    print('layer ', idx, ' Object  : ', layer)
    print('      ', idx, ' Id      : ', layer.Id)
    print('      ', idx, ' Index   : ', layer.Index)
    print('      ', idx, ' Name    : ', layer.Name)
    print('      ', idx, ' Fullpath: ', layer.FullPath)
    print('      ', idx, ' Color   : ', layer.Color)
    print()

layer table object:  <rhino3dm._rhino3dm.File3dmLayerTable object at 0x10c83c5b0>
num of layers:  6

layer  0  Object  :  <rhino3dm._rhino3dm.Layer object at 0x108476130>
       0  Id      :  212960d6-ad46-439d-8516-772ce85fde03
       0  Index   :  0
       0  Name    :  0
       0  Fullpath:  0
       0  Color   :  (0, 0, 0, 255)

layer  1  Object  :  <rhino3dm._rhino3dm.Layer object at 0x10844e530>
       1  Id      :  25488de8-55a5-470b-a098-ea3c8c1220b3
       1  Index   :  1
       1  Name    :  ! cable
       1  Fullpath:  ! cable
       1  Color   :  (255, 191, 0, 255)

layer  2  Object  :  <rhino3dm._rhino3dm.Layer object at 0x10c84ee30>
       2  Id      :  6f40c8fc-3378-4efb-a590-0af0bf5a85dc
       2  Index   :  2
       2  Name    :  ! num
       2  Fullpath:  ! num
       2  Color   :  (0, 127, 0, 255)

layer  3  Object  :  <rhino3dm._rhino3dm.Layer object at 0x10c84e1b0>
       3  Id      :  4af54445-51c1-4fff-b520-54055554debc
       3  Index   :  3
       3  Name    : 

## read `ObjectsTable`

In [5]:
object_table = file3dm_obj.Objects
print('object table: ', object_table, '\n')

object table:  <rhino3dm._rhino3dm.File3dmObjectTable object at 0x10c84db30> 


## create dataclass `OatPV`

In [6]:
@dataclass
class OatPV:
    idx: int
    outline: rhino3dm.PolylineCurve
    str_num: str = ''
    id: str = field(default_factory=generate_id)

## read `pv`

In [7]:
layer_idx_of_pv = 4
cnt_pv = 0
max_num_obj = 3
total_num_pv = 0

oatPV_list = []

for idx, obj in enumerate(iter(object_table)):
    if (obj.Attributes.LayerIndex == layer_idx_of_pv) and (cnt_pv < max_num_obj):
        print('object ', idx, ' Object                : ', obj)
        print('       ', idx, ' Geometry              : ', obj.Geometry)
        print('       ', idx, ' Attributes            : ', obj.Attributes)
        print('       ', idx, ' Attributes.Id         : ', obj.Attributes.Id)
        print('       ', idx, ' Attributes.Name       : ', obj.Attributes.Name)
        print('       ', idx, ' Attributes.LayerIndex : ', obj.Attributes.LayerIndex)
        print('       ', idx, ' Attributes.ObjectColor: ', obj.Attributes.ObjectColor)
        print('       ', idx, ' Attributes.Visible    : ', obj.Attributes.Visible)
        print()

        # if geom is PolylineCurve, print its details
        if isinstance(obj.Geometry, rhino3dm.PolylineCurve):
            print('            curve IsClosed     :', obj.Geometry.IsClosed)
            print('            curve IsPlanar     :', obj.Geometry.IsPlanar())
            print('            curve IsPolyline   :', obj.Geometry.IsPolyline())
            print()

            print('            curve PointCount   :', obj.Geometry.PointCount)
            
            for i in range(obj.Geometry.PointCount):
                print('            curve pt ', i, ' : ', obj.Geometry.Point(i))
                # print('           curve pt ', i, ' : ', obj.Geometry.ToPolyline().PointAt(i)) -- alternative

            print()

            print('            curve SegmentCount :', obj.Geometry.ToPolyline().SegmentCount)

            for i in range(obj.Geometry.ToPolyline().SegmentCount):
                print('            segement ', i, ' : ', obj.Geometry.ToPolyline().SegmentAt(i))

            print()

            oatPV = OatPV(idx, obj.Geometry)
            oatPV_list.append(oatPV)

        cnt_pv += 1
    else:
        pass

for idx, obj in enumerate(iter(object_table)):
    if (obj.Attributes.LayerIndex == layer_idx_of_pv):
        total_num_pv += 1

print('total num of pv: ', total_num_pv)
print()

for idx, oatPV in enumerate(oatPV_list):
    print('oatPV ', idx, ' : ', oatPV)

object  72  Object                :  <rhino3dm._rhino3dm.File3dmObject object at 0x10c80cdb0>
        72  Geometry              :  <rhino3dm._rhino3dm.PolylineCurve object at 0x10c87f570>
        72  Attributes            :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x10c87f570>
        72  Attributes.Id         :  36f08c55-5986-4aa2-b0b8-ae56b26b043c
        72  Attributes.Name       :  
        72  Attributes.LayerIndex :  4
        72  Attributes.ObjectColor:  (0, 0, 0, 255)
        72  Attributes.Visible    :  True

            curve IsClosed     : True
            curve IsPlanar     : True
            curve IsPolyline   : True

            curve PointCount   : 5
            curve pt  0  :  35.10614556790845,209.9589626310085,0.0
            curve pt  1  :  35.28014104193071,210.94573999952675,0.0
            curve pt  2  :  36.95467106254545,210.65047517688606,0.0
            curve pt  3  :  36.78067558852319,209.6636978083678,0.0
            curve pt  4  :  35.10614556790845

## read `cable`

In [8]:
layer_idx_of_cable = 1
cnt_cable = 0
max_num_obj = 3
total_num_cable = 0

for idx, obj in enumerate(iter(object_table)):
    # print the first 3 obj on a layer
    if (obj.Attributes.LayerIndex == layer_idx_of_cable) and (cnt_cable < max_num_obj):
        print('object ', idx, ' Object                : ', obj)
        print('       ', idx, ' Geometry              : ', obj.Geometry)
        print('       ', idx, ' Attributes            : ', obj.Attributes)
        print('       ', idx, ' Attributes.Id         : ', obj.Attributes.Id)
        print('       ', idx, ' Attributes.Name       : ', obj.Attributes.Name)
        print('       ', idx, ' Attributes.LayerIndex : ', obj.Attributes.LayerIndex)
        print('       ', idx, ' Attributes.ObjectColor: ', obj.Attributes.ObjectColor)
        print('       ', idx, ' Attributes.Visible    : ', obj.Attributes.Visible)
        print()

        # if geom is PolylineCurve, print its details
        if isinstance(obj.Geometry, rhino3dm.PolylineCurve):
            print('            curve IsClosed     :', obj.Geometry.IsClosed)
            print('            curve IsPlanar     :', obj.Geometry.IsPlanar())
            print('            curve IsPolyline   :', obj.Geometry.IsPolyline())
            print()

            print('            curve PointCount   :', obj.Geometry.PointCount)
            
            for i in range(obj.Geometry.PointCount):
                print('            curve pt ', i, ' : ', obj.Geometry.Point(i))
                # print('           curve pt ', i, ' : ', obj.Geometry.ToPolyline().PointAt(i)) -- alternative

            print()

            print('            curve SegmentCount :', obj.Geometry.ToPolyline().SegmentCount)

            for i in range(obj.Geometry.ToPolyline().SegmentCount):
                print('            segement ', i, ' : ', obj.Geometry.ToPolyline().SegmentAt(i))

            print()

        cnt_cable += 1
    else:
        pass

for idx, obj in enumerate(iter(object_table)):
    if (obj.Attributes.LayerIndex == layer_idx_of_cable):
        total_num_cable += 1

print('total num of cable: ', total_num_cable)

object  0  Object                :  <rhino3dm._rhino3dm.File3dmObject object at 0x10c83c4f0>
        0  Geometry              :  <rhino3dm._rhino3dm.PolylineCurve object at 0x108474bb0>
        0  Attributes            :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x108474bb0>
        0  Attributes.Id         :  fa8bcce9-953e-47f0-bcfc-4de7a87e0c76
        0  Attributes.Name       :  
        0  Attributes.LayerIndex :  1
        0  Attributes.ObjectColor:  (0, 0, 0, 255)
        0  Attributes.Visible    :  True

            curve IsClosed     : False
            curve IsPlanar     : True
            curve IsPolyline   : True

            curve PointCount   : 6
            curve pt  0  :  14.298569362095407,251.0734261769187,0.0
            curve pt  1  :  2.232971074135598,253.20091370662107,0.0
            curve pt  2  :  1.9377067622771804,251.5263865827965,0.0
            curve pt  3  :  14.003304539454732,249.398896156304,0.0
            curve pt  4  :  13.700564187398314,247.

## read `text`

In [9]:
layer_idx_of_text = 2
cnt_text = 0
max_num_obj = 3
total_num_text = 0

for idx, obj in enumerate(iter(object_table)):
    if (obj.Attributes.LayerIndex == layer_idx_of_text) and (cnt_text < max_num_obj):
        print('object ', idx, ' Object                  : ', obj)
        print('       ', idx, ' Geometry                : ', obj.Geometry)
        print('       ', idx, ' Attributes              : ', obj.Attributes)
        print('       ', idx, ' Attributes.Id           : ', obj.Attributes.Id)
        print('       ', idx, ' Attributes.Name         : ', obj.Attributes.Name)
        print('       ', idx, ' Attributes.LayerIndex   : ', obj.Attributes.LayerIndex)
        print('       ', idx, ' Attributes.ObjectColor  : ', obj.Attributes.ObjectColor)
        print('       ', idx, ' Attributes.Visible      : ', obj.Attributes.Visible)
        print()

        # if geom is AnnotationBase, print its PlainText
        if isinstance(obj.Geometry, rhino3dm.AnnotationBase):
            print('            Geometry.PlainText     : ', obj.Geometry.PlainText)
            print()

        cnt_text += 1

for idx, obj in enumerate(iter(object_table)):
    if (obj.Attributes.LayerIndex == layer_idx_of_text):
        total_num_text += 1

print('total num of text: ', total_num_text)

object  4944  Object                  :  <rhino3dm._rhino3dm.File3dmObject object at 0x10c8aff70>
        4944  Geometry                :  <rhino3dm._rhino3dm.Text object at 0x10c84ea70>
        4944  Attributes              :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x10c84ea70>
        4944  Attributes.Id           :  c9b6923f-bb4c-4d0c-8407-9c230a7f9d5c
        4944  Attributes.Name         :  
        4944  Attributes.LayerIndex   :  2
        4944  Attributes.ObjectColor  :  (0, 0, 0, 255)
        4944  Attributes.Visible      :  True

            Geometry.PlainText     :  6-6-1

object  4945  Object                  :  <rhino3dm._rhino3dm.File3dmObject object at 0x10c8b4ef0>
        4945  Geometry                :  <rhino3dm._rhino3dm.Text object at 0x10c8aff70>
        4945  Attributes              :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x10c8aff70>
        4945  Attributes.Id           :  cef51aca-20e2-495b-97e2-6e0e1c5b8494
        4945  Attributes.Name      

## create dataclass `MyPt`

In [10]:
@dataclass
class OatPt:
    idx  : int
    point: rhino3dm.Point3d
    name : str

## read `pt`

``` python
# layer_idx_of_pt = 2
# cnt_pt = 0
# max_num_obj = 3
# total_num_pt = 0
#
# oaPt_list_1 = []
#
# for idx, obj in enumerate(iter(object_table)):
#     if (obj.Attributes.LayerIndex == layer_idx_of_pt) and (cnt_pt < max_num_obj):
#         print('object ', idx, ' Object                  : ', obj)
#         print('       ', idx, ' Geometry                : ', obj.Geometry)
#         print('       ', idx, ' Attributes              : ', obj.Attributes)
#         print('       ', idx, ' Attributes.Id           : ', obj.Attributes.Id)
#         print('       ', idx, ' Attributes.Name         : ', obj.Attributes.Name)
#         print('       ', idx, ' Attributes.LayerIndex   : ', obj.Attributes.LayerIndex)
#         print('       ', idx, ' Attributes.ObjectColor  : ', obj.Attributes.ObjectColor)
#         print('       ', idx, ' Attributes.Visible      : ', obj.Attributes.Visible)
#         print()
#
#         # if geom is Point, print its xyz
#         if isinstance(obj.Geometry, rhino3dm.Point):
#             pt = obj.Geometry.Location
#             print('            Geometry.Location.X     : ', pt.X)
#             print('            Geometry.Location.Y     : ', pt.Y)
#             print('            Geometry.Location.Z     : ', pt.Z)
#
#             oatPt = OatPt(idx, pt, str(idx))
#             oaPt_list_1.append(oatPt)
#
#             print()
#
# for idx, obj in enumerate(iter(object_table)):
#     if (obj.Attributes.LayerIndex == layer_idx_of_pt):
#         total_num_pt += 1
#
# print('total num of pt: ', total_num_pt)
```

In [11]:
# for idx, oatPt in enumerate(oaPt_list_1):
#     print('myPt', idx, ' : ', oatPt)
#     print('    type of idx in myPt:', type(oatPt.idx))
#     print('    type of point in myPt:', type(oatPt.point))
#     print('    type of name in myPt:', type(oatPt.name))

## read `TextDot`

In [12]:
max_num_obj = 3
cnt_textDot = 0
total_num_textDot = 0

for idx, obj in enumerate(iter(object_table)):
    obj_geom = obj.Geometry
    obj_attr = obj.Attributes

    if (isinstance(obj_geom, rhino3dm.TextDot)) and (cnt_textDot < max_num_obj):
        print('textDot object ', idx, ' : ', obj_geom)
        print('        position            : ', obj_geom.Point)
        print('        text                : ', obj_geom.Text)
        print('        Attributes object   : ', obj_attr)
        print('        attribs.Id          : ', obj_attr.Id)
        print('        attribs.Name        : ', obj_attr.Name)
        print('        attribs.LayerIndex  : ', obj_attr.LayerIndex)
        print('        attribs.ObjectColor : ', obj_attr.ObjectColor)
        print('        attribs.Visible     : ', obj_attr.Visible)
        print()

        cnt_textDot += 1

for idx, obj in enumerate(iter(object_table)):
    if isinstance(obj.Geometry, rhino3dm.TextDot):
        total_num_textDot += 1

print('total num of textDot: ', total_num_textDot)

textDot object  4872  :  <rhino3dm._rhino3dm.TextDot object at 0x10c8af230>
        position            :  27.064173777436537,200.67848337106952,0.0
        text                :  6-6-1
        Attributes object   :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x10c87ebb0>
        attribs.Id          :  ace09291-2363-4c79-a99d-2ae7be61531d
        attribs.Name        :  
        attribs.LayerIndex  :  3
        attribs.ObjectColor :  (0, 0, 0, 255)
        attribs.Visible     :  True

textDot object  4873  :  <rhino3dm._rhino3dm.TextDot object at 0x10c84dfb0>
        position            :  35.21171531476992,205.36180830113673,0.0
        text                :  1-1-1
        Attributes object   :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x10c8af230>
        attribs.Id          :  b7bc5562-82b9-4119-b8ef-aa9c68ce3dee
        attribs.Name        :  
        attribs.LayerIndex  :  3
        attribs.ObjectColor :  (0, 0, 0, 255)
        attribs.Visible     :  True

textDot object 

## read `EarthAnchorPoint`

In [13]:
# get the `Settings` object as a property of File3dm object
file3dm_settings = file3dm_obj.Settings 
print('file3dm settings: ', file3dm_settings)

# get `EarthAnchorPoint` in settings
earthAnchorPoint = file3dm_settings.EarthAnchorPoint
print('EarthAnchorPoint: ', earthAnchorPoint)
print()

# check if the earthAnchorPoint is set or not
earthAnchorPoint_set_or_not = earthAnchorPoint.EarthLocationIsSet()
print('earthAnchorPoint is set: ', earthAnchorPoint_set_or_not)

# get lat and lon of earthAnchorPoint
earthAnchorPoint_lat = earthAnchorPoint.EarthBasepointLatitude
earthAnchorPoint_lon = earthAnchorPoint.EarthBasepointLongitude
print('earthAnchorPoint latitude : ', earthAnchorPoint_lat)
print('earthAnchorPoint longitude: ', earthAnchorPoint_lon)
print()

# get ModelBasePoint
modelBasePoint = earthAnchorPoint.ModelBasePoint
print('model base point: ', modelBasePoint)

# get `ModelToEarthTransform`
modelUnitSystem = file3dm_settings.ModelUnitSystem
print('modelUnitSystem : ', modelUnitSystem)

modelToEarthTransform = earthAnchorPoint.GetModelToEarthTransform(modelUnitSystem)
print('modelToEarthTransform : ', modelToEarthTransform)

# create a test pt
# test_pt = rhino3dm.Point3d(91.2724, 187.784, 0)
test_pt = rhino3dm.Point3d(140.492,236.638,0)
test_pt_transformed_in_lon_lat_ele = test_pt.Transform(modelToEarthTransform)
print('test pt transformed in lon, lat, ele: ', test_pt_transformed_in_lon_lat_ele)

file3dm settings:  <rhino3dm._rhino3dm.File3dmSettings object at 0x10c8ae5b0>
EarthAnchorPoint:  <rhino3dm._rhino3dm.EarthAnchorPoint object at 0x108458eb0>

earthAnchorPoint is set:  True
earthAnchorPoint latitude :  22.716611
earthAnchorPoint longitude:  120.534701

model base point:  1.887214762712837,254.12517650890035,0.0
modelUnitSystem :  UnitSystem.Meters
modelToEarthTransform :  <rhino3dm._rhino3dm.Transform object at 0x10c87e8f0>
test pt transformed in lon, lat, ele:  120.53605149715442,22.716453831047513,0.0

> **File3dm.Settings**
>
> [***property***
> `rhino3dm.File3dm.Settings`](https://mcneel.github.io/rhino3dm/python/api/File3dm.html#rhino3dm.File3dm.Settings)
>
> <br />
>
> File3dmSettings: Settings include tolerance, and unit system, and
> defaults used for creating views and objects.
>
> <br />
>
> Return: A rhino3dm.File3dm.Settings object

> **rhino3dm.File3dmSettings**
>
> [***class***
> `rhino3dm.File3dm`](https://mcneel.github.io/rhino3dm/python/api/File3dmSettings.html)
>
> <br />
>
> Property:
>
> -   `ModelBasePoint`
>
> -   `EarthAnchorPoint`
>
> -   `ModelUnitSystem`

> **rhino3dm.EarthAnchorPoint**
>
> [***class***
> `rhino3dm.EarthAnchorPoint`](https://mcneel.github.io/rhino3dm/python/api/EarthAnchorPoint.html#module-rhino3dm)
>
> Property:
>
> -   EarthBasepointLatitude
>
> -   EarthBasepointLongitude
>
> -   ModelBasePoint
>
> -   GetModelToEarthTransform(modelUnitSystem)

> **Set EarthAnchorPoint in Rhino**
>
> [***cmd***
> `Set EarthAnchorPoint in Rhino`](https://docs.mcneel.com/rhino/5/help/en-us/commands/earthanchorpoint.htm)

> **Rhino to Google Earth**
>
> [***wiki***
> `Rhino to Google Earth`](https://wiki.mcneel.com/rhino/googleearth)